# HDInsight, PIG

In [2]:
from pyquickhelper.ipythonhelper import add_notebook_menu
add_notebook_menu()

## Download the data

In [3]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00222/"
file = "bank.zip"
import pyensae
data = pyensae.download_data(file, website=url)

In [74]:
import pandas
df = pandas.read_csv("bank-full.csv", sep=";")

In [75]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [76]:
df.to_csv("bank_full_tab_no.txt", sep="\t", index=False, header=None)

## Connect to the cluster

In [3]:
import pyensae
blobstorage = 
blobpassword = 
hadoop_server = 
hadoop_password = 
username = "centrale"
client, bs =  %hd_open
client, bs

(<pyensae.remote.azure_connection.AzureClient at 0x1c3711b72e8>,
 <azure.storage.blob.blockblobservice.BlockBlobService at 0x1c3711b7320>)

## Upload the data

In [10]:
%blob_up bank_full_tab.txt hdblobstorage/centrale/bank_full_tab_no.txt

'centrale/bank_full_tab_no.txt'

In [11]:
%blob_ls hdblobstorage/centrale

,name,last_modified,content_type,content_length,blob_type
0,centrale/bank_full.csv,2016-06-15 22:17:59+00:00,None,4610348,BlockBlob
1,centrale/bank_full_tab.txt,2016-06-15 22:19:46+00:00,None,3751306,BlockBlob
2,centrale/bank_full_tab_no.txt,2016-06-15 23:00:52+00:00,None,3751306,BlockBlob


## Submit a PIG query

In [77]:
mapping = {'int64': 'double', 'float': 'double', 'object': 'chararray'}
schema = ["%s:%s" % (_[0], mapping.get(str(_[1]), _[1])) for _ in zip(df.columns, df.dtypes)]
schema = ", ".join(schema)
schema

'age:double, job:chararray, marital:chararray, education:chararray, default:chararray, balance:double, housing:chararray, loan:chararray, contact:chararray, day:double, month:chararray, duration:double, campaign:double, pdays:double, previous:double, poutcome:chararray, y:chararray'

In [78]:
%%PIG_azure aggage2.pig
values = LOAD '$CONTAINER/centrale/bank_full_tab_no.txt' USING PigStorage('\t') AS (age:double, 
                                                    job:chararray, marital:chararray, education:chararray, 
                                                   default:chararray, balance:double, housing:chararray, loan:chararray, 
                                                   contact:chararray, day:double, month:chararray, duration:double, 
                                                   campaign:double, 
                                                   pdays:double, previous:double, poutcome:chararray, y:chararray);
gr = GROUP values BY loan ;
agg = FOREACH gr GENERATE group, AVG(age) AS avg_age ;
STORE agg INTO '$CONTAINER/centrale/bank_full_tab_no_agg.txt' USING PigStorage('\t') ;

In [79]:
jid = %hd_pig_submit aggage2.pig

In [80]:
jid

{'id': 'job_1466026353703_0009'}

In [81]:
%hd_queue

[{'detail': None, 'id': 'job_1466026353703_0009'},
 {'detail': None, 'id': 'job_1466026353703_0008'},
 {'detail': None, 'id': 'job_1466026353703_0007'},
 {'detail': None, 'id': 'job_1466026353703_0006'},
 {'detail': None, 'id': 'job_1466026353703_0005'},
 {'detail': None, 'id': 'job_1466026353703_0004'},
 {'detail': None, 'id': 'job_1466026353703_0003'},
 {'detail': None, 'id': 'job_1466026353703_0002'},
 {'detail': None, 'id': 'job_1466026353703_0001'}]

In [88]:
df = %hd_job_status jid['id']
df["status"]["state"]

'RUNNING'

In [90]:
%blob_ls /centrale

,name,last_modified,content_type,content_length,blob_type
0,centrale/bank_full.csv,2016-06-15 22:17:59+00:00,None,4610348,BlockBlob
1,centrale/bank_full_tab.txt,2016-06-15 22:19:46+00:00,None,3751306,BlockBlob
2,centrale/bank_full_tab_no.txt,2016-06-15 23:00:52+00:00,None,3751306,BlockBlob
3,centrale/scripts/pig/aggage.pig,2016-06-15 23:15:54+00:00,None,782,BlockBlob
4,centrale/scripts/pig/aggage.pig.log,2016-06-15 23:16:40+00:00,None,0,BlockBlob
5,centrale/scripts/pig/aggage.pig.log/exit,2016-06-15 23:16:40+00:00,None,3,BlockBlob
6,centrale/scripts/pig/aggage.pig.log/stderr,2016-06-15 23:16:30+00:00,None,4060,BlockBlob
7,centrale/scripts/pig/aggage.pig.log/stdout,2016-06-15 23:16:30+00:00,None,0,BlockBlob
8,centrale/scripts/pig/aggage2.pig,2016-06-15 23:20:24+00:00,None,806,BlockBlob
9,centrale/scripts/pig/aggage2.pig.log,2016-06-15 23:21:10+00:00,None,0,BlockBlob


In [91]:
%hd_tail_stderr -n 200 jid['id']